In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from math import sqrt
from xgboost import plot_importance
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [2]:
tagsCount = pd.read_csv("SMPdata/imageIDandTagsCount.csv")
#tagsCount

### test other type tags count

for i in range(len(tagsCount)):
    tagsCount.loc[i,"TagsCount"] = round(sqrt(tagsCount.loc[i,"TagsCount"]))
    if i%3000==0:
        print(i)
tagsCount

In [3]:
imageInformation = pd.read_csv("SMPdata/image_information.txt",skiprows=[765,768]).astype("int")
#print(imageInformation)

In [4]:
label = pd.read_csv("train_all_json/train_label.txt")
#print(label)

In [5]:
trainData = pd.concat( [tagsCount , label], axis=1 )
#print(trainData)

In [6]:
trainData = pd.merge(imageInformation, trainData, how='left', on='imageID')
#print(trainData.shape)

In [7]:
#print(trainData.head())
#print(tagsCount.head())
#print(imageInformation.head())

In [8]:
qooSortData = pd.read_csv("SMPdata/train.csv")

#print(label.loc[1])
#print(qooSortData)
#print(label.loc[tagsCount[tagsCount["imageID"]==385070026].index])

In [9]:
trainDataII = qooSortData[["image","Uid","Concept"]]
#print(trainDataII)
trainDataII["Uid_code"] = LabelEncoder().fit_transform(trainDataII["Uid"])
trainDataII["Concept_code"] = LabelEncoder().fit_transform(trainDataII["Concept"])
trainDataII.drop(columns = ["Uid","Concept"],inplace=True)
trainDataII.columns = ["imageID","Uid_code","Concept_code"]
print(trainDataII)
#qooSortData

            imageID  Uid_code  Concept_code
0         385070026     28628           254
1         943750056      2619           200
2        3246928439     13086           105
3        3432316502     30463           426
4        3538960847      1752           602
5        3544037429      1752           212
6        3544739218      1752           212
7        3544861110      1752           212
8        3544891702      1752           212
9        4298739132     30462           170
10       4387678456      8006           230
11       4497890817      7600           121
12       4539636819     10235           490
13       4539636893     10235           490
14       4539636935     10235           490
15       4539637115     10235           490
16       4539637159     10235           490
17       4539637185     10235           490
18       4539637207     10235           490
19       4539637259     10235           490
20       4539637389     10235           490
21       4539637413     10235   

/anaconda3/envs/testtensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/testtensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/envs/testtensorflow/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [10]:
train = pd.merge(trainDataII, trainData, how="inner", on="imageID")
#train

detectObject = pd.read_csv("SMPdata/imageDetectObject.csv",index_col = "Unnamed: 0")
train = pd.merge(train, detectObject, how="inner", on="imageID")
train

In [11]:
a , X_test= train_test_split(train, test_size=0.2)
X_train , X_valid = train_test_split(a,test_size=0.1)
Y_train = X_train["score"]
X_train = X_train.drop(["score"],axis = 1)
Y_test = X_test["score"]
X_test = X_test.drop(["score"],axis = 1)
Y_valid = X_valid["score"]
X_valid = X_valid.drop(["score"],axis = 1)

In [12]:
param_test1 = { 'max_depth':range(7,9,1), 'min_child_weight':range(1,3,1) } 
#param_test1 = {'n_estimators':[50, 100, 200, 500,1000],'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3]}
"""gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, 
                                                  n_estimators=140, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8, 
                                                  objective= 'binary:logistic', 
                                                  nthread=4, 
                                                  scale_pos_weight=1, 
                                                  seed=27), 
                        param_grid = param_test1, 
                        scoring='roc_auc',
                        n_jobs=4,iid=False, cv=5)"""
gsearch1 = GridSearchCV(estimator = XGBRegressor(booster='gbtree',
                    objective= 'reg:linear',
                    eval_metric='rmse',
                    gamma = 0.1,
                    min_child_weight= 2,
                    max_depth= 9,
                    subsample= 0.8,
                    colsample_bytree= 0.8,
                    tree_method= 'exact',
                    learning_rate=0.1,
                    n_estimators=1000,
                    nthread=4,
                    scale_pos_weight=1,
                    seed=27),
                       param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,Y_train) 
gsearch1.cv_results_ , gsearch1.best_params_, gsearch1.best_score_

/anaconda3/envs/testtensorflow/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/testtensorflow/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


({'mean_fit_time': array([101.25049458, 100.49673047, 117.86549225, 111.37129402]),
  'std_fit_time': array([0.40516261, 0.55326747, 4.79478425, 3.20507041]),
  'mean_score_time': array([1.68594818, 1.20531011, 1.36696038, 1.11537957]),
  'std_score_time': array([0.16562084, 0.11662349, 0.08213114, 0.21835799]),
  'param_max_depth': masked_array(data=[7, 7, 8, 8],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_min_child_weight': masked_array(data=[1, 2, 1, 2],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_depth': 7, 'min_child_weight': 1},
   {'max_depth': 7, 'min_child_weight': 2},
   {'max_depth': 8, 'min_child_weight': 1},
   {'max_depth': 8, 'min_child_weight': 2}],
  'split0_test_score': array([-0.2317834 , -0.2311273 , -0.21898774, -0.21934173]),
  'split1_test_score': array([-0.2318943 , -0.232793  , -0.22019379, -0.22217926]),
  'split2_